
## Extract information from images.
## Graphs are taken from camelcamelcamel website


In [1]:
# First attempt:
import os
from glob import glob
import json
from PIL import Image, ImageEnhance
import numpy as np
import datetime
import dill
import pytesseract
import matplotlib.pyplot as plt


In [ ]:
filenames = [os.path.basename(i)[:-4] for i in glob('data/price_history/new/*.png',)]
filenames =  glob('data/price_history/new/*.png')
print len(filenames)
fname = filenames[2]

In [ ]:
# factor = 2
# im_ = im1
# # print im_.height, im_.width
# im3 = im_.resize(( im_.width * factor,im_.height * factor),Image.BICUBIC)
# show(im3)
# im3.show()
# # im3.save('year.png')

# boxes = pytesseract.image_to_string(im3,  boxes=True, config = '-c tessedit_char_whitelist=0123456789')
# # boxes = pytesseract.image_to_string(im3,  boxes=True, config = '--user-words dic.txt')

# print boxes

In [2]:
class camelScrap(object):
    def __init__(self, fname, alpha = 1.5):
        if os.path.exists(fname):
            self.im = Image.open(fname)
            self.fname = fname
            self.asin = fname[-14:-4]
            _im1 = self.im.crop((0,455,970,470))
            _im1 = ImageEnhance.Contrast(_im1)
            _im1 = _im1.enhance(alpha)
            self.timeStrip = _im1
            self.isempty = (np.array(self.timeStrip)<255).sum()<100
            self._boxes = ''
            self._t1 = 0
            self._t2 = 0
            self._Scale = None
            self.label = None
            self.y_zero = None
            self.extracted_new = None
            self.factor= 2
            self.x_end = 0
            
        else:
            print 'wrong filename, obejct was not created'
    def __str__(self):
        return 'member of camelScrap: ASIN %s, dimensions %d x %d' %(self.asin, self.im.width, self.im.height)
    def __repr__(self):
        return 'member of camelScrap: ASIN %s' %(self.asin)
    
    def show(self, scale=0.1):
        plt.figure(figsize=(20,10))
        plt.imshow(self.im)
    

    def OCR_time(self, factor = 2):
        self.factor = factor
        im_ = self.timeStrip
        im_ = im_.resize(( im_.width * factor,im_.height * factor),Image.BICUBIC)
        self.years = pytesseract.image_to_string(im_,  boxes=False, config = '--psm 7 -c tessedit_char_whitelist=0123456789') 
        self._boxes = pytesseract.image_to_string(im_,  boxes=True, config = '--psm 7 -c tessedit_char_whitelist=0123456789')
        
        return self.years
    def setCoordinates(self):
        if self._boxes =='': self.OCR_time()            
        boxes = self._boxes
        factor = self.factor
        a = np.array([a.split(' ') for a in boxes.split('\n')])
        s = ''.join(a[:,0])
        if len(s)>16:
            ##calculate last recorded full year range (so not the padding years) 
            year_begin = int(s[4:8])
            year_end = 2017

            _ind = s.find(str(year_end))

            if year_end != year_begin + (1.*_ind/4-1):
                print 'end year does not match'

        #             if self.years.split()[-4:-1] !=['2015','2016', '2017']: ## same as
                if s[_ind-4:_ind] == str(year_end-1) and s[_ind-8:_ind-4] == str(year_end-2):
                    year_begin = year_end -(_ind/4-1)
                else:
                    print "failed twice !"
                    print '2016 is',s[_ind-4:_ind] == str(year_end-1) , '2015 is',s[_ind-8:_ind-4] == str(year_end-2)


            # x1 = 1.*a[4,1]/factor
            x_begin = float(a[4,1])/factor

            x_end = float(a[_ind,1])/factor
            self.x_end = x_end

            t2 = int(datetime.datetime(year_end,01,01).strftime("%s"))
            t1 = int(datetime.datetime(year_begin,01,01).strftime("%s"))
            Scale = 1.0*(t2-t1)/(x_end-x_begin)
            self._t1 = t1
            self._t2 = t2
            self._Scale = Scale
        else:
            self._Scale = 0
            print 'Only very recent data!'
        
    def getTime(self,pixel):
        if self._Scale ==None: self.setCoordinates()
        t1,t2,Scale = self._t1,self._t2, self._Scale
        t = t2 + int((pixel - self.x_end)*Scale)
        t_datetime = datetime.date.fromtimestamp(t)
        return t, t_datetime
    
    def find_zero(self):
        ## returns estimated y-coordinate for y==0(in pixels from the left top corner) 
        ## it does that by searching for a dark grey line that is persistent across all image

        if self.isempty: return
        ima = np.array(self.im)
        Ydim, Xdim = ima.shape[:2]
        print 'hello'
        print np.all(ima[:,400,:]<75, axis=1)
        foo = [np.where(np.all(ima[:,col,:]==75, axis=1))[0] for col in range(Xdim)]
        print foo[:5]
        foo = [[n,i.mean()] for n,i in enumerate(foo) if  len(i)]
        foo = np.array(foo)
        print foo[:5]
        y_zero = int(np.median(foo))
        self.y_zero = y_zero
        return y_zero
    def find_new(self):
        ima = np.array(self.im)
        Ydim, Xdim = ima.shape[:2]
        if self.y_zero is None: self.find_zero()
        y_zero = self.y_zero
        foo = [np.where((ima[:y_zero,col,0]<10)&(ima[:y_zero,col,2]>150))[0] for col in range(Xdim)]
        foo = [[n, y_zero - i.mean()] for n,i in enumerate(foo) if  len(i)]
        extracted_new = np.array(foo)  
        extracted_new[:,0] = [self.getTime(pixel)[0] for pixel in extracted_new[:,0]]
        self.extracted_new = extracted_new
        return extracted_new
    def OCR_getLabel(self):
        im = self.im.crop((45,0,545,16)).resize((1000,30), Image.BILINEAR)
        self.label = pytesseract.image_to_string(im,  boxes=False, lang = 'eng', config = r'--psm 7 -c tessedit_char_blacklist=\\//')
        
    def populate(self):
        if  self.isempty: 
            print 'Seem like there is no plot'
            return
        if self.label == None:
            self.OCR_getLabel()
        if self.extracted_new is None:
            self.find_new()
        
def UtoD(i):
#     Just to remmeber hpw to go from Unix timestamp to date object and back
    return datetime.datetime.fromtimestamp(i)
def DtoU(i):
#     Just to remmeber hpw to go from Unix timestamp to date object and back
    return int(i.strftime('%s'))


In [ ]:
print fname
a = camelScrap(filenames[counter-1])
a.find_zero()
# (np.array(a.im)==75).sum()
# [np.where(np.all(np.array(a.im)[:,col,:1]==75, axis=1))[0] for col in range(1000)]

In [ ]:
fname = filenames[0]
# fname = r'data/price_history/new/B0012ZMBDQ.png'
a = camelScrap(fname,1.5)
print a
a.timeStrip

In [ ]:
a.populate()
# a.__dict__

In [ ]:
plt.plot(*a.extracted_new[:800,:].T)


In [3]:
from IPython.display import clear_output
tag = 'new'
path = 'data/price_history/%s/' % tag
filenames =  glob('data/price_history/new/??????????.png')

    
counter = 0
for fname in  filenames:
    counter +=1
    path_to_save = fname[:-4]+'.json'
    if not os.path.exists(path_to_save):
        a = camelScrap(fname)
        if not a.isempty:
            clear_output(wait =True)
            a.populate()
            print counter,'/',len(filenames), '>>' ,a.label
        else:
            a.extracted_new = np.array([0,0])
    
        out = {'data': a.extracted_new.tolist(), 'asin':a.asin, 'label_OCR': a.label}
        json.dump(out, open(path_to_save, 'w'), indent=4)
        
#         or dill:
#         out = {'data':a.extracted_new, 'asin':a.asin, 'label_OCR': a.label}
#         dill.dump(out, open(path_to_save, 'w'))
    
    
        
    
    
    
#     plt.plot(*a.extracted_new[:800,:].T)
#     plt.show()
    
    

hello
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False Fa

/Users/vitya/.virtualenvs/py27/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/vitya/.virtualenvs/py27/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: cannot convert float NaN to integer

In [ ]:
im3 = a.im.crop((45,0,545,16)).resize((1000,34), Image.BILINEAR)
im3

In [ ]:
pytesseract.image_to_string(im3,  boxes=False, lang = 'eng', config = r'--psm 7 -c tessedit_char_blacklist=\\//')
# pytesseract.image_to_string(im3,  boxes=False, lang = 'eng', config = r'--psm 7 -c tessedit_char_whitelist=qwertyuiopasdfghjklzxcvbnM-1234567890')

In [ ]:
print fname
filenames[counter-1]

In [ ]:
a = camelScrap(filenames[counter-1])
a.populate()


In [ ]:
a.setCoordinates()
a.find_new()

In [ ]:
a.__dict__

In [ ]:
a.find_new()


In [ ]:
plt.plot(a.extracted_new[:,0],a.extracted_new[:,1])


In [ ]:
a.__dict__

In [ ]:
# im2 = Image.new('RGBA', (50,30), (255,255,255,0))
# im2.paste(image.crop((30,424,90,455)))
# im2 = im2.rotate(-45,expand = False,resample=Image.BICUBIC)
# plt.imshow(im2)
N = 8
for i, fname in enumerate(filenames[:N]):
    
    image = Image.open(fname)
    # image.crop((10,20,30,40))
    # image.show()
    plt.subplot(int(np.ceil(8**.5)), int(np.ceil(8**.5)), i+1)
#     plt.imshow(image)
    im2 = Image.new('RGBA', (55,35), (255,255,255,0))
    im2.paste(image.crop((15,424,100,455)))
    im2 = im2.rotate(-45,expand = False,resample=Image.BICUBIC)
#     im2 = im2.convert('L') # convert image to monochrome - this works
    im2 = im2.resize((im2.width*2,im2.height*2))
    im2 = ImageEnhance.Contrast(im2)
    im2 = im2.enhance(2)

    plt.imshow(im2)

    
#     display(im2)

In [ ]:


## temp
path = 'data/price_history/new/B002D89VHK.json'

im = Image.open(path[:-4]+'png')


df = pd.read_json(path)
# json.load(open(path,'r'))
im

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib
pdf = pd.DataFrame(np.array([i for i in df.data]))
pdf.columns = ['Time', 'Value']
pdf.Time =  [datetime.date.fromtimestamp(i) for i in pdf.Time]
# pdf.Time = pd.to_datetime(pdf.Time, unit= 's')
# pdf.plot()
pdf.head()
plt.plot(pdf.Time, pdf.Value)
plt.xticks(rotation=45)
# pdf.dtypes